# rf601_intminuit
Likelihood and minimization: interactive minimization with MINUIT




**Author:** Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Saturday, November 16, 2024 at 10:25 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooProdPdf.h"
#include "RooAddPdf.h"
#include "RooMinimizer.h"
#include "RooFitResult.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"
using namespace RooFit;

Setup pdf and likelihood
-----------------------------------------------

Observable

In [2]:
RooRealVar x("x", "x", -20, 20);

Model (intentional strong correlations)

In [3]:
RooRealVar mean("mean", "mean of g1 and g2", 0);
RooRealVar sigma_g1("sigma_g1", "width of g1", 3);
RooGaussian g1("g1", "g1", x, mean, sigma_g1);

RooRealVar sigma_g2("sigma_g2", "width of g2", 4, 3.0, 6.0);
RooGaussian g2("g2", "g2", x, mean, sigma_g2);

RooRealVar frac("frac", "frac", 0.5, 0.0, 1.0);
RooAddPdf model("model", "model", RooArgList(g1, g2), frac);

[#0] WARNING:InputArguments -- The parameter 'sigma_g1' with range [-inf, inf] of the RooGaussian 'g1' exceeds the safe range of (0, inf). Advise to limit its range.


Generate 1000 events

In [4]:
std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};

input_line_52:2:2: warning: 'data' shadows a declaration with the same name in the 'std' namespace; use '::data' to reference this declaration
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
 ^


Construct unbinned likelihood of model w.r.t. data

In [5]:
std::unique_ptr<RooAbsReal> nll{model.createNLL(*data)};

input_line_53:2:51: error: reference to 'data' is ambiguous
 std::unique_ptr<RooAbsReal> nll{model.createNLL(*data)};
                                                  ^
input_line_52:2:30: note: candidate found by name lookup is 'data'
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
                             ^
/usr/include/c++/9/bits/range_access.h:318:5: note: candidate found by name lookup is 'std::data'
    data(initializer_list<_Tp> __il) noexcept
    ^
/usr/include/c++/9/bits/range_access.h:289:5: note: candidate found by name lookup is 'std::data'
    data(_Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:299:5: note: candidate found by name lookup is 'std::data'
    data(const _Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:309:5: note: candidate found by name lookup is 'std::data'
    data(_Tp (&__array)[_Nm]) noexcept
    ^


Interactive minimization, error analysis
-------------------------------------------------------------------------------

Create MINUIT interface object

In [6]:
RooMinimizer m(*nll);

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { _ZN5cling7runtime8internal15DynamicExprInfoC1EPKcPPvb, _ZN12__cling_N5241mE, __orc_init_func.cling-module-312, _ZN12__cling_N52424__dynamic__cling_Un1Qu30E, __cxx_global_var_initcling_module_312_.2, _GLOBAL__sub_I_cling_module_312, _ZN5cling7runtime8internal15DynamicExprInfoC2EPKcPPvb, $.cling-module-312.__inits.0, __cxx_global_var_initcling_module_312_, _ZNK5cling7runtime8internal15LifetimeHandler9getMemoryEv, _ZN12__cling_N52416__cling_Un1Qu323EPv }) }
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime8internal15LifetimeHandlerC1EPNS1_15DynamicExprInfoEPN5clang11DeclContextEPKcPNS_11InterpreterE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::internal::LifetimeHandler::LifetimeHandler(cling::runtime::internal::DynamicExprInfo*, clang::DeclContext*, char const*, cling::Interpreter*)
Maybe you need to load the corresponding shared library?

Activate verbose logging of MINUIT parameter space stepping

In [7]:
m.setVerbose(true);

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __orc_init_func.cling-module-312 }) }
cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Call MIGRAD to minimize the likelihood

In [8]:
m.migrad();

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Print values of all parameters, that reflect values (and error estimates)
that are back propagated from MINUIT

In [9]:
std::unique_ptr<RooArgSet>{model.getParameters(x)}->Print("s");

  1) RooRealVar::     frac = 0.5
  2) RooRealVar::     mean = 0
  3) RooRealVar:: sigma_g1 = 3
  4) RooRealVar:: sigma_g2 = 4


Disable verbose logging

In [10]:
m.setVerbose(false);

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12RooMinimizer10setVerboseEb }) }


Run HESSE to calculate errors from d2L/dp2

In [11]:
m.hesse();

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Print value (and error) of sigma_g2 parameter, that reflects
value and error back propagated from MINUIT

In [12]:
sigma_g2.Print();

RooRealVar::sigma_g2 = 4  L(3 - 6) 


Run MINOS on sigma_g2 parameter only

In [13]:
m.minos(sigma_g2);

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Print value (and error) of sigma_g2 parameter, that reflects
value and error back propagated from MINUIT

In [14]:
sigma_g2.Print();

RooRealVar::sigma_g2 = 4  L(3 - 6) 


Saving results, contour plots
---------------------------------------------------------

Save a snapshot of the fit result. This object contains the initial
fit parameters, the final fit parameters, the complete correlation
matrix, the EDM, the minimized FCN , the last MINUIT status code and
the number of times the RooFit function object has indicated evaluation
problems (e.g. zero probabilities during likelihood evaluation)

In [15]:
std::unique_ptr<RooFitResult> fitResult{m.save()};

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { _ZNSt11_Tuple_implILm0EJP12RooFitResultSt14default_deleteIS0_EEE7_M_headERS4_, _ZNSt10unique_ptrI12RooFitResultSt14default_deleteIS0_EE11get_deleterEv, __orc_init_func.cling-module-321, $.cling-module-321.__inits.0, _ZNSt15__uniq_ptr_implI12RooFitResultSt14default_deleteIS0_EEC2EPS0_, _ZNSt15__uniq_ptr_implI12RooFitResultSt14default_deleteIS0_EE6_M_ptrEv, _ZN12__cling_N53316__cling_Un1Qu332EPv, _ZSt3getILm0EJP12RooFitResultSt14default_deleteIS0_EEERNSt13tuple_elementIXT_ESt5tupleIJDpT0_EEE4typeERS8_, _GLOBAL__sub_I_cling_module_321, _ZNSt10_Head_baseILm0EP12RooFitResultLb0EE7_M_headERS2_, _ZNSt10unique_ptrI12RooFitResultSt14default_deleteIS0_EEC1IS2_vEEPS0_, _ZNSt11_Tuple_implILm1EJSt14default_deleteI12RooFitResultEEEC2Ev, _ZSt12__get_helperILm0EP12RooFitResultJSt14default_deleteIS0_EEERT0_RSt11_Tuple_implIXT_EJS4_DpT1_EE, _

Make contour plot of mx vs sx at 1,2,3 sigma

In [16]:
RooPlot *frame = m.contour(frac, sigma_g2, 1, 2, 3);
frame->SetTitle("Minuit contour plot");

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { _ZN12__cling_N53416__cling_Un1Qu333EPv, __cxx_global_var_initcling_module_322_, __orc_init_func.cling-module-322, _ZN12__cling_N5345frameE, _GLOBAL__sub_I_cling_module_322, $.cling-module-322.__inits.0 }) }


Print the fit result snapshot

In [17]:
fitResult->Print("v");

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __orc_init_func.cling-module-321 }) }
cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5339fitResultE }) }


Change parameter values, floating
-----------------------------------------------------------------

At any moment you can manually change the value of a (constant)
parameter

In [18]:
mean = 0.3;

Rerun MIGRAD,HESSE

In [19]:
m.migrad();
m.hesse();
frac.Print();

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Now fix sigma_g2

In [20]:
sigma_g2.setConstant(true);

Rerun MIGRAD,HESSE

In [21]:
m.migrad();
m.hesse();
frac.Print();

new TCanvas("rf601_intminuit", "rf601_intminuit", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();

cling JIT session error: Failed to materialize symbols: { (main, { _ZN12__cling_N5241mE }) }


Draw all canvases 

In [22]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()